In [1]:
# Install libraries required
# !pip install langchain --user
# !pip install pypdf
# !pip install pinecone-client
# !pip install openai
# !pip install tiktoken

Create Account on [OpenAI](https://platform.openai.com/) and [Pinecone](https://www.pinecone.io/) to get the API Key

In [2]:
# Load libraries
import os 
import pinecone
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

C:\Users\alif\anaconda3\envs\test\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
C:\Users\alif\anaconda3\envs\test\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\alif\anaconda3\envs\test\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\alif\anaconda3\envs\test\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [13]:
# Define secret API key and variable from above platforms
OPENAI_KEY = ""
PINECONE_KEY = ""
ENVIRONMENT_NAME = ""

In [4]:
# Set environment variable of OPENAI Key
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

In [5]:
# Initialize pinecone
pinecone.init(
    api_key = PINECONE_KEY,
    environment = ENVIRONMENT_NAME
)

In [6]:
# Create index - Custom Knowledge Base.
loader = PyPDFLoader("dataset/Machine_Learning_Operations_MLOps__Overview,_Definition,_and_Architecture.pdf")
pages = loader.load_and_split()

In [7]:
# Split text from PDF into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

docs = text_splitter.split_documents(pages)

In [8]:
# Create embedding based on OpenAIEmbedding technique
embeddings = OpenAIEmbeddings()

In [9]:
# Initialize document searching based on index in Pinecone
index_name = "test-index"
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [10]:
# define LLM and QA Object
llm = OpenAI(temperature=0.2)
qa = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = docsearch.as_retriever(search_kwargs={"k": 2})
)

In [12]:
# Test Query based on our Custom Knowledge Base
query = "What is MLOps?"
qa.run(query)

'\n\nMLOps (also known as DevOps for machine learning) is a practice that combines software development and operations processes to automate and streamline the process of building, testing, and deploying machine learning models. It helps to ensure that models are developed and deployed in a consistent and reliable way.'